In [47]:
import pytesseract
from PIL import Image
import os

# Path to the Tesseract executable (change this according to your setup)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Users\USER\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
ocr_tesseract_path = os.path.join(os.getcwd(), "tesseract_ocr\\tesseract.exe")
pytesseract.pytesseract.tesseract_cmd = ocr_tesseract_path

def parse_receipt(image_path):
    with Image.open(image_path) as img:
        text = pytesseract.image_to_string(img)
        return text

# Path to your receipt image
image_path = os.path.join(os.getcwd(), "sample_receipts\\QFC_1.jpg")
receipt_text = parse_receipt(image_path)



In [19]:
receipt_text

'2746.NE ;\n506 504-0 4 i Street\n\nYour ( ashiee was CHEC 522\n\nIWIN ie MILK\nnett ae\n. J\ney QF SAVINGS 0 ee\ns Mega Fvent payings!. 00\nsc PROGRESSO <t 1.49 F\nPi OFC USAIN : 2. mo\nVv\n\nFX NEXCARI E TAPE\n\nTAP! 6.49 X\nDAVES RITER BREADQA 4,99 F\nGre Say INS 2.00\nWI GRAPES. blak\n1.75 Ib @ 3,49 /\nWT GRAPES BLACK\ngit tb ¢ ron.\n.99 /Ib\nWI NAVE\n; DORIS\n1. 28\n\nLS CA\n28 Ib @ 2.99 /Ib\nWI bas) al Hones\n\nos\n\nSTRWBRY\n0.80 |\n\nwo @ wr\nOo WwW OD O— Ww\n\nWT RONA TOMATO\nEGGPLANT\n.64 Ib @ 1, a /\\b\nso panes\n202.9\n\n“Ayncg HASS 5.98 F\nCALB CRUNCHY Coe 1.99 F\nf Sc Meg mtr Savings! . 00\n= CALB | LENT II 1 SNAP as 1990 F\nS¢ Meya Event Savings! .00\n’ CAB LENTIL SNAP. di 1999 F\nse Mega Event Savings1.00\nCALB LENTIL SNAPS <4 1.99 F\n\nwont\nHue ee a Sa) ge\n\nmn oo :\n\nws\nseeks\na\n>\n=\n\nSc Mega Event Savin naa 1.00\n\nsc CALB CRUNCHY we Si 1,99 F\n\nOre leva Bat Savings! . 00\n\nst ADVANTAGE (ISTOMER 44444446176\nFue Paine\n\naning Points\nel bus ‘ai Points\n\ni 

In [55]:

print(get_chat_response(receipt_text))

{
  "store_name": "2776.NE",
  "items": [
    {
      "name": "IWIN ie MILK",
      "price": 0.0,
      "quantity": 1
    },
    {
      "name": "PROGRESSO",
      "price": 1.49,
      "quantity": 1
    },
    {
      "name": "DAVES RITER BREADQA",
      "price": 4.99,
      "quantity": 1
    },
    {
      "name": "GRAPES BLACK",
      "price": 1.75,
      "quantity": 0.5
    },
    {
      "name": "GRAPES BLACK",
      "price": 0.99,
      "quantity": 1
    },
    {
      "name": "ONA TOMATO EGGPLANT",
      "price": 0.64,
      "quantity": 2.9
    },
    {
      "name": "Ayncg HASS",
      "price": 5.98,
      "quantity": 1
    },
    {
      "name": "CALB CRUNCHY Coe",
      "price": 1.99,
      "quantity": 1
    },
    {
      "name": "CALB LENT II SNAP",
      "price": 1.99,
      "quantity": 1
    },
    {
      "name": "CALB LENTIL SNAP",
      "price": 1.99,
      "quantity": 4
    },
    {
      "name": "CALB CRUNCHY we Si",
      "price": 1.99,
      "quantity": 1
    }
  ],

In [53]:
template = '''
{
  "store_name": "{{store_name}}",
  "store_address": "{{store_name}}",
  "items": [
    {
      "name": "{{item1_name}}",
      "price": {{item1_price}},
      "quantity": {{item1_quantity}}
    },
    {
      "name": "{{item2_name}}",
      "price": {{item2_price}},
      "quantity": {{item2_quantity}}
    }
  ],
  "date": "{{date}}",
  "total": {{total}}
}
'''



In [54]:

import openai
from decouple import config
from config import Config

openai.organization = Config.OPEN_AI_ORG
openai.api_key = Config.OPEN_AI_KEY
client = openai.OpenAI()


def get_chat_response(message_input):

    messages = [{"role": "system", "content": "You will parse out store name, items (name, price, quantity), date, total from the user receipt and print as this format: " + template}]
    user_message = {"role": "user", "content": message_input}
    messages.append(user_message)

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        message_text = response.choices[0].message.content
        return message_text
    except Exception as e:
        print(e)
        return

In [45]:
import base64
import requests

# OpenAI API Key
api_key = Config.OPEN_AI_KEY

# Function to encode the image

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = os.path.join(os.getcwd(), "sample_receipts\\QFC_1.jpg")

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4-turbo",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "In the receipt, parse out item name, item price, and item quantity as a csv."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9LZiK2LIOpg8tQT7CnmhQITrXlS48', 'object': 'chat.completion', 'created': 1714929200, 'model': 'gpt-4-turbo-2024-04-09', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Here's the list of items from the receipt presented in CSV format (item name, item price, item quantity):\n\n```\nLumindoork, Milk, $6.29\nPhil, Soup, $2.19\nBig, Dippers, $2.16\nOrganic, Blueberries, $2.00\nHoneycrisp, Apples, $1.00\nDeav', Saltines, $4.99\nGrapes, Black, $8.13\nGrapes, Black, $2.89\nEggplant, $2.89\nNavel, Oranges, $6.00\nNavel, Oranges, $3.83\nGreen, Onions, $2.39\nRed, Potatoes, $3.24\nGarlic, $2.74\nAvacado, Hass, $5.98\nCereal, Crunchy Loops, $1.99\nPears, $1.00\nAvo,* Dev, $1.99\nCajun, Snaps, $1.99\nCajun, Snaps, $1.99\nAvocado, Alvacados, $1.99\nRusset, Potatoes, $4.99\nOranges, Seed, $4.99\n0.820, lbs, $1.70\nBanana, $1.50\nOnion, $0.49\n"}, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 1131, 'completion_tokens': 300, 'total_toke

In [46]:
a = response.json()
print(a['choices'][0]['message']['content'])

Here's the list of items from the receipt presented in CSV format (item name, item price, item quantity):

```
Lumindoork, Milk, $6.29
Phil, Soup, $2.19
Big, Dippers, $2.16
Organic, Blueberries, $2.00
Honeycrisp, Apples, $1.00
Deav', Saltines, $4.99
Grapes, Black, $8.13
Grapes, Black, $2.89
Eggplant, $2.89
Navel, Oranges, $6.00
Navel, Oranges, $3.83
Green, Onions, $2.39
Red, Potatoes, $3.24
Garlic, $2.74
Avacado, Hass, $5.98
Cereal, Crunchy Loops, $1.99
Pears, $1.00
Avo,* Dev, $1.99
Cajun, Snaps, $1.99
Cajun, Snaps, $1.99
Avocado, Alvacados, $1.99
Russet, Potatoes, $4.99
Oranges, Seed, $4.99
0.820, lbs, $1.70
Banana, $1.50
Onion, $0.49

